In [1]:
import librosa
import soundfile as sf
import numpy as np
import os
import random

In [5]:
COMPRESSION_FACTOR = 0.4
LIST_OF_WORDS = []  # Read from .csv file fra Johan

In [ ]:
def reverse_audio(in_path, word):
    """Reverse the audio"""
    y, sr = librosa.load(in_path, sr=None)
    rev = y[::-1]  # Reverse the audio array
    out_path = f"reverse/{word}_reversed.wav"
    sf.write(out_path, rev, sr)
    return out_path

for word in LIST_OF_WORDS:
    in_path = f"original_sound/{word}.wav"
    out_path_reversed = reverse_audio(in_path, word)
    print(f"Reversed audio for word: {word} saved to {out_path_reversed}.")

In [8]:
def time_compress(in_path, factor=COMPRESSION_FACTOR):
    """Time-compress by factor"""
    y, sr = librosa.load(in_path, sr=None)
    y_fast = librosa.effects.time_stretch(y, rate=1 + factor)
    out_path = f"primes/{word}_compressed.wav"
    sf.write(out_path, y_fast, sr)
    return out_path

for word in LIST_OF_WORDS:
    in_path = f"original_sound/{word}.wav"
    out_path_compressed = time_compress(in_path)
    print(f"Time-compressed audio for word: {word} saved to {out_path_compressed}.")

In [ ]:
def create_silence(duration_ms, sr=22050):
    """Create silence array"""
    duration_sec = duration_ms / 1000.0
    samples = int(duration_sec * sr)
    return np.zeros(samples)

def concatenate_audio(audio_files, silence_duration_ms=100):
    """Concatenate audio files with silence between them"""
    sr = None
    audio_segments = []
    
    for audio_file in audio_files:
        y, current_sr = librosa.load(audio_file, sr=sr)
        if sr is None:
            sr = current_sr
        audio_segments.append(y)
        if audio_file != audio_files[-1]:  # Add silence except after last file
            audio_segments.append(create_silence(silence_duration_ms, sr))
    
    combined = np.concatenate(audio_segments)
    return combined, sr

In [ ]:
def create_trial(word):
    """Create one trial: random masks from other primes"""
    prime = f"primes/{word}_compressed.wav"
    others = [w for w in os.listdir("reverse") if w != f"{word}_reversed.wav"]
    forward_mask_word = random.choice(others)
    backward_mask_word = random.choice([w for w in others if w != forward_mask_word])


    # Combine sequence
    audio_files = [forward_mask_path, prime_compressed_path, backward_mask_path]
    combined, sr = concatenate_audio(audio_files, silence_duration_ms=100)

    out_path = os.path.join(OUTPUT_DIR, f"{word}_trial.wav")
    sf.write(out_path, combined, sr)
    return out_path, forward_mask_path, backward_mask_path

In [ ]:
base_paths = "original_sound"

all_masks = []

for file in os.listdir("original_sound"):
    trial, fwd, bwd = create_trial(file, base_paths)
    all_masks.extend([fwd, bwd])
    print(f"✅ Trial for '{file}' created → {trial}")